<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/main/notebooks/financials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [ ]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/financials"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

#### Setup and Configuration

In [ ]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [ ]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [ ]:
# @title Dropdown Menus
import ipywidgets as widgets

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [ ]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [ ]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Plots & Reports

In [ ]:
# @title _ {"run":"auto"}
source = "U.S. Listed Stocks" #@param ["U.S. Listed Stocks", "Taiwan Stocks"]
backend = "Plotly" # @param ["mplfinance","Plotly"]

import yfinance as yf

import ipywidgets as widgets
from IPython.display import display, clear_output

import vistock.stock_indices as si
import vistock.tw as tw
import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

MAX_SELECTIONS = 1

src_codes = {
    'U.S. Listed Stocks': 'USLS',
    'Taiwan Stocks': 'TWSE+TPEX+ESB'
}[source]
symbols = si.get_tickers(src_codes)
if source == 'Taiwan Stocks':
    symbols = [f"{tw.stock_name(s)} {s}" for s in symbols]
dropdowns, layout = create_search_dropdowns(symbols, MAX_SELECTIONS)

theme_dropdown = {
    'mplfinance': create_style_dropdown('Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Theme:', 'plotly'),
}[backend]

btn_plot = widgets.Button(description="Plot a Chart")
btn_report = widgets.Button(description="Generate Reports")
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
outputs = widgets.VBox()

display(
    layout, theme_dropdown,
    widgets.HBox([btn_plot, btn_report]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
)

chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def get_symbols():
    selected = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        selected = [s.split()[0] for s in selected]
    return selected

def on_plot_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No stock selected. Please select up to 8 stocks.")
        elif backend == 'mplfinance':
            chart.plot(symbol=selected[0], style=theme_dropdown.value)
        else:  # Plotly
            chart.plot(symbol=selected[0],template=theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No stock selected. Please select up to 8 stocks.")
        else:
            symbol = selected[0]
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot.on_click(on_plot_click)
btn_report.on_click(on_report_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
